<a href="https://colab.research.google.com/github/gnapat/CNN_Imageclassified/blob/DecodePrediction/presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
print( f"Python {sys.version}\n" )

import numpy as np
print( f"NumPy {np.__version__}\n" )

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
print( f"Matplotlib {matplotlib.__version__}\n" )

import tensorflow as tf
print( f"TensorFlow {tf.__version__}" )
print( f"tf.keras.backend.image_data_format() = {tf.keras.backend.image_data_format()}" )

# Count the number of GPUs as detected by tensorflow
gpus = tf.config.list_physical_devices('GPU')
print( f"TensorFlow detected { len(gpus) } GPU(s):" )
for i, gpu in enumerate(gpus):
  print( f".... GPU No. {i}: Name = {gpu.name} , Type = {gpu.device_type}" )

import os
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import MobileNet

import cv2 as cv

Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]

NumPy 1.23.5

Matplotlib 3.7.1

TensorFlow 2.14.0
tf.keras.backend.image_data_format() = channels_last
TensorFlow detected 0 GPU(s):


Dataset preparing

In [2]:
!ls -lrt
!cp drive/MyDrive/Gz/nida/dl/hw01/THFOOD50-v1.zip ./
!unzip THFOOD50-v1.zip

Streaming output truncated to the last 5000 lines.
 extracting: THFOOD50-v1/train/PadPakRuamMit/057.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/058.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/059.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/060.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/061.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/062.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/064.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/065.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/066.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/067.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/068.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/069.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/070.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/071.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/072.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/073.jpg  
  inflating: THFOOD50-v1/train/PadPakRuamMit/074.jpg  
  inflating: T

In [3]:
def to_array(X):
  images=[]
  for x in X:
    img_array = tf.keras.preprocessing.image.img_to_array(x).astype(np.uint8)
    images.append(img_array)

  X_ret = np.array(images)

  return X_ret




def do_augmentation(img,y,aug):

  ret_x=[]
  ret_y=[]

  if aug == 1:
    aug_list = [Image.FLIP_LEFT_RIGHT]
  elif aug == 2:
    aug_list = [Image.ROTATE_180]
  elif aug == 3:
    #aug_list = [Image.ROTATE_90,Image.ROTATE_180,Image.ROTATE_270]
    aug_list = [Image.ROTATE_90,Image.ROTATE_90,Image.ROTATE_90]
  elif aug == 4:
    #aug_list = [Image.ROTATE_90,Image.ROTATE_180,Image.ROTATE_270,Image.FLIP_LEFT_RIGHT,Image.ROTATE_90,Image.ROTATE_180,Image.ROTATE_270]
    aug_list = [Image.ROTATE_90,Image.ROTATE_90,Image.ROTATE_90,Image.FLIP_LEFT_RIGHT,Image.ROTATE_90,Image.ROTATE_90,Image.ROTATE_90]



  ret_x.append(img)
  ret_y.append(y)

  #img = tf.keras.utils.array_to_img(img)

  for iaug in aug_list:
    img = img.transpose(method=iaug)
    #img_arr = tf.keras.preprocessing.image.img_to_array(img)
    ret_x.append(img)
    ret_y.append(y)



  return ret_x,ret_y

def load_dataset(xlist,ylist,aug=0):

  images = []
  labels = []

  cc = 0
  for f in xlist:
    img = tf.keras.preprocessing.image.load_img(f, target_size=(224, 224))

    if aug != 0:
      # do aug
      img_aug , y_aug = do_augmentation(img=img,y=ylist[cc],aug=aug)

      images.extend(img_aug)
      labels.extend(y_aug)
      #pass

    else:
      #img = tf.keras.preprocessing.image.img_to_array(img)
      img = tf.keras.preprocessing.image.img_to_array(img)
      images.append(img)
      labels.append(ylist[cc])

    cc += 1

  return images,labels




def prepare_dataset(path,lables,test_size=0.2, random_state=42,split=True,augmentation=0,do_img_array=0):

  x = []
  y=[]
  x_path=[]
  images = []
  X_train = np.empty((244,244,3))
  X_test = np.array([])
  y_train = np.empty( (244,244,3))
  y_test =np.array([])

  for i in rock_label:
    folder_path=f"{path}/{rock_label[i]}"
    print(folder_path)
    files = os.listdir(folder_path)
    for file in files:
      target_path = f"{folder_path}/{file}"
      #print(target_path)
      #img =  tf.keras.preprocessing.image.load_img(target_path, target_size=(224, 224))
      #img = tf.keras.preprocessing.image.img_to_array(img)
      #images.append(img)

      x_path.append(target_path)
      y.append(int(i))

  if split == True:
    X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(x_path, y, test_size=0.3, random_state=42)
    #print(type(X_train_temp))
    X_train_temp, y_train_temp = load_dataset(X_train_temp,y_train_temp,augmentation)
    #print(type(X_train_temp))
    X_test_temp, y_test_temp = load_dataset(X_test_temp,y_test_temp,augmentation)

  if split == True:
    if do_img_array == 0:
      # return with image format
      return X_train_temp, X_test_temp, y_train_temp, y_test_temp
    else:
      # return with nunpy array
      X_train_temp = to_array(X_train_temp)
      X_test_temp = to_array(X_test_temp)
      y_train_temp = np.array(y_train_temp)
      y_test_temp = np.array(y_test_temp)

      return X_train_temp, X_test_temp, y_train_temp, y_test_temp
  else:
    return X,Y


In [4]:
path="THFOOD50-v1/train"
rock_label={"0":"Joke","1":"KaiThoon","2":"KaiJeowMooSaap","3":"KaoManGai"}

X_train,X_test,Y_train,Y_test = prepare_dataset(path=path,lables=rock_label,split=True,augmentation=3,do_img_array=1)


THFOOD50-v1/train/Joke
THFOOD50-v1/train/KaiThoon
THFOOD50-v1/train/KaiJeowMooSaap
THFOOD50-v1/train/KaoManGai


In [5]:
base_architecture = {"VGG16": VGG16(weights="imagenet", include_top=False),
                     "ResNet50":ResNet50(weights='imagenet', include_top=False),
                     "MobileNet":MobileNet(weights='imagenet', include_top=False)}

processing= {"VGG16": tf.keras.applications.vgg16,
                     "ResNet50":tf.keras.applications.resnet50,
                     "MobileNet":tf.keras.applications.mobilenet_v3}

base_architecture_list = ["VGG16","ResNet50","MobileNet"]


94765736/94765736 [==============================] - 1s 0us/step


17225924/17225924 [==============================] - 0s 0us/step


In [6]:
list_pred=[]
for m in base_architecture_list:
  print(m)
  model = base_architecture[m]
  x_train_preprocess = processing[m].preprocess_input(X_train)
  x_test_preprocess = processing[m].preprocess_input(X_test)

  img_preprocess1 = np.expand_dims(x_train_preprocess[2000],axis=0)
  pred = model.predict(img_preprocess1)
  list_pred.append(pred)

VGG16
1/1 [==============================] - 1s 813ms/step
ResNet50
1/1 [==============================] - 1s 851ms/step
MobileNet
1/1 [==============================] - 0s 384ms/step
